In [1]:
import requests
from tqdm import tqdm
import shutil

headers = {"User-Agent": "My User Agent"}

In [8]:
url = input()

In [15]:
response = requests.get(url, stream=True, headers=headers)
with open('example.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)